## 一、Tomcat优化思路梳理
* 无限制的增大业务线程。
* 优化请求队列的数量。
* 第一，根据服务器情况，来制定业务线程池的数量和请求队列的数量；第二，如果业务线程和请求队列一定的情况下，让我们的业务线程处理的更快；第三，优化线程，思路:a.加大它的资源（内存更多或CPU更多），b.对线程模型的优化。

## Connector配置建议
### 线程池优化
* `maxConnections`：最大连接数。
    1. 最大连接数是受系统内核的影响。`Linux`的系统内核数，可以通过`ulimit －a`的方式查看服务器上允许的最大连接数。修改`/etc/security/limits.conf`下，把`core`和`rss`两项进行放开，把这两个都变成`nofiles`，且对应的值都是**65535**。
    2. 添加maxConnections的配置。（配置完成以后，一定要进行压测！超过Tomcat最大承载能力的时候，性能会出现急剧降低的现象。）
        2.1 **对CPU要求更高时**，建议不要配置过于大；
        2.2 **对CPU要求不是特别高时**，建议配置在**3000左右**。
    3. 配置的地方:`server.xml`中找`connector`。
* `maxThreads`：最大线程数。服务器上**500－700是相对合理的数量**。配置的位置也是在`server.xml中的connector`里面进行配置。
* `acceptCount`:最大排队等待数（也就是前面我们提到的请求队列）。**（单Tomcat能处理的最大数量就是`maxThreads`与`acceptCount`的数量之和）**。`一般建议与最大线程数持平或略低就可以接受。`

### Tomcat内存优化（能使每一个连接处理的速度更快）
JVM优化建议

参数|参数作用|优化建议
:-:|:-:|:-:
-server|启用Serveer|服务端建议开启
－Xms|最小内存|建议与－Xmx相同
－Xmx|最大内存|建议到可用内存的80%
－XX:MetaspaceSize|元空间初始值|
-XX:MaxMetaspaceSize|元空间最大内存|默认无限
-XX:MaxNewSize|新生代最大内存|默认16M

* JVM在最大内存和最小内存进行漂移的过程中，本身是浪费资源的。当最大内存和最小内存配置一样的时候，JVM就不会来回漂移。
* 上面的第4和第5是专门针对1.8及以上的。（以前是持久内存）
* 修改的位置在`tomcat/bin/catalina.sh`。通过添加`JAVA_OPTS="-server -Xms128m -Xmx128m -XX:MetaspaceSize=128m -XX:MaxMetaspaceSize=128m -XX:MaxNewSize=32m"`
* 验证的方式是通过启动后，通过`jmap -heap [进程编号]`的命令进行查看。

### Tomcat其他内容优化
* Gzip相关配置(**修改的位置在conf/server.xml中的Connector上**)
    * `compression`:设置开启Gzip压缩(True或者False)。
    * `compressableMimeType`:压缩类型()。
    * `compressMinSize`:压缩后输出内容大小(2048等具体的值)。
* 其他项配置
    * `enableLookups`:开启反查域名(某些场景下比较好用，但是一般还是不用开启)。
    * `connectionTimeout`:网络连接超时阈值。
    * `minSpareThreads`:最小空闲线程数(**没有访问的情况下保持多少活跃**)。